### Programa para adquisisción en los experimentos de transitorios en circuitos RC, RL y RLC

Estamos estudiando fenómenos transitorios en los circuitos RC, RL o RLC.

Tenemos dos curvas en el osciloscopio y las queremos adquirir para análizar los datos.

En ese contexto, este programa muestra una guía paso a paso para hacer la adquisición.

Es importante notar que este programa sirve para trabajar con los dos modelos de osciloscopios disponibles en el laboratorio, TDS1052B-EDU y TBS1000C, prestar atención que una de las celdas tienen que indicar con que modelo estan trabajando. Por que el comando utilizado en esa celda es levemente diferente para un equipo respecto del otro. El resto de los comandos es exáctamente igual para ambos modelos.

Empezamos importando las librerías a utilizar:

In [ ]:
import pyvisa as visa
import time
import numpy as np
import pandas as pd
from matplotlib import pyplot as plt

Abrimos el Resource Manager de **pyvisa** para conectarnos con el equipo:

In [ ]:
rm = visa.ResourceManager()

lista_instrumentos = rm.list_resources()  

print(lista_instrumentos)

Seleccionamos el instrumento y establecemos comunicación:

In [ ]:
instrumento = ''

osci=rm.open_resource()

print(osci.query('*IDN?'))

Formateamos los datos que vamos a recibir:

In [ ]:
osci.write('DAT:ENC RPB') 
osci.write('DAT:WID 1') 
osci.write('HEADER OFF')

Congelamos la pantalla antes de comenzar a adquirir:

In [ ]:
osci.write('ACQ:STATE OFF')

Tomamos las escalas en ambos canales. 

**ATENCIÓN!** Aquí hay que indicar el modelo de osciloscopio que tienen para que el programa pueda a travez de un bloque *if* decidir cuales son los comandos a utilizar:

In [ ]:
modelo_osci = "TDS1052B"   # aca poner el modelo del osciloscopio que tengan : "TDS1052B" o  "TBS1000C"


if modelo_osci=="TDS1052B":

    xze1, xin1, yze1, ymu1, yoff1 = osci.query_ascii_values('WFMPRE:CH1:XZE?;XIN?;YZE?;YMU?;YOFF?;',separator=';')
    time.sleep(2)
    xze2, xin2, yze2, ymu2, yoff2 = osci.query_ascii_values('WFMPRE:CH2:XZE?;XIN?;YZE?;YMU?;YOFF?;',separator=';')



elif modelo_osci=="TBS1000C":

    xze1, xin1, yze1, ymu1, yoff1 = osci.query_ascii_values('WFMOUTPRE:CH1:XZE?;XIN?;YZE?;YMU?;YOFF?;',separator=';')
    time.sleep(2)
    xze2, xin2, yze2, ymu2, yoff2 = osci.query_ascii_values('WFMOUTPRE:CH2:XZE?;XIN?;YZE?;YMU?;YOFF?;',separator=';')


Seleccionamos el **canal 1** como fuente de datos y adquirimos la curva:

In [ ]:
osci.write('DAT:SOU CH1' )

dataCH1 = osci.query_binary_values('CURV?', datatype='B',container=np.array)

Análogamente, seleccionamos el **canal 2** como fuente de datos y adquirimos la curva:

In [ ]:
osci.write('DAT:SOU CH2')    

dataCH2=osci.query_binary_values('CURV?', datatype='B',container=np.array)

Descongelamos la pantalla y cerramos conexión con el osci (si es que no lo van a utilizar más):

In [ ]:
osci.write('ACQ:STATE ON')

osci.close()

Pasamos los datos a magnitudes físicas, tiempo en *segundos* y voltaje en *Volts*:

In [ ]:
t1 = xze1 + np.arange(len(dataCH1))*xin1 
V1=(dataCH1-yoff1)*ymu1+yze1 

t2 = xze2 + np.arange(len(dataCH2))*xin2 
V2=(dataCH2-yoff2)*ymu2+yze2 

Hacemos un ploteo rápido para chequear que los datos adquiridos se corresponden con lo que realmente queremos:

In [ ]:
fig, ax  =  plt.subplots()

ax.plot(t1, V1, label="V CH1")

ax.plot(t2, V2, label="V CH2")

ax.set_xlabel('Tiempo [s]')
ax.set_ylabel('Voltaje [V]')

plt.show()

Guardamos los datos para hacer un análisis posterior:

In [ ]:
data = { "t1":t1, "V1":V1, "t2":t2, "V2":V2 }
df = pd.DataFrame( data=data ) 

carpeta = "C:path\to\you\folder"
df.to_csv( carpeta+"Transitorio_Curvas.csv", index=False )

df.head()   